In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torchvision.transforms as transforms

from torch.nn import Module
from torch.nn import Conv1d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten

import pandas as pd

from helpers.models.helper import accuracy, accuracy20

In [2]:
class CNN(Module):
    def __init__(self, numChannels=20):
        # call the parent constructor
        super(CNN, self).__init__()
        self.conv1 = Conv1d(in_channels=numChannels, out_channels=15, kernel_size=15, padding=7)
        self.relu1 = nn.Sigmoid()
        self.conv2 = Conv1d(in_channels=15, out_channels=10, kernel_size=15, padding=7)
        self.relu2 = nn.Sigmoid()
        self.conv3 = Conv1d(in_channels=10, out_channels=5, kernel_size=15, padding=7)
        self.relu3 = nn.Sigmoid()
        self.conv4 = Conv1d(in_channels=5, out_channels=2, kernel_size=15, padding=7)
        self.relu4 = nn.Sigmoid()
        self.conv5 = Conv1d(in_channels=2, out_channels=1, kernel_size=15, padding=7)
        self.sigmoid = nn.Sigmoid()
        # self.logSoftmax = LogSoftmax(dim=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.conv4(x)
        x = self.relu4(x)
        x = self.conv5(x)
        x = self.sigmoid(x)
        # output = self.logSoftmax(x)
        return x

In [3]:
class SequenceDataset(Dataset):
    def __init__(self, transform):
        self.data = pd.read_pickle('../data/cnn/data.csv')
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # x = input[index].T # channels should be first
        # y = output[index].reshape((1, -1)) # make the data match the shape of X after passing through all layers
        x = self.data['in'].iloc[index]
        # if index == 1:
        #     print(x.shape)
        # x = x.T
        y = self.data['out'].iloc[index]
        y = y.reshape((1,-1))

        if self.transform:
            x = self.transform(x)[0]

        return x, y

In [4]:
# input = np.random.uniform(low=0, high=1, size=(100, 100, 20))
# output = np.random.uniform(low=0, high=1, size=(100, 100))

In [5]:
dataset = SequenceDataset(transforms.ToTensor())
length = len(pd.read_pickle('../data/cnn/data.csv'))
train_len = (length * 9) // 10
test_len = length - train_len
train_set, test_set = torch.utils.data.random_split(dataset, [train_len, test_len])

train_loader = DataLoader(dataset=train_set, batch_size=1, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=1, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CNN().double()

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [6]:
for epoch in range(3):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, (inputs, target) in enumerate(train_loader):


        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        target = target.reshape(-1)
        outputs = outputs.reshape(-1)
        # print(target)
        # print()
        # print(outputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 0.452
[1,  4000] loss: 0.403
[1,  6000] loss: 0.394
[1,  8000] loss: 0.388
[2,  2000] loss: 0.359
[2,  4000] loss: 0.332
[2,  6000] loss: 0.308
[2,  8000] loss: 0.307
[3,  2000] loss: 0.294
[3,  4000] loss: 0.291
[3,  6000] loss: 0.287
[3,  8000] loss: 0.279
Finished Training


In [7]:
# # test
with torch.no_grad():
    n_samples = 0
    total_acc = 0
    for input, target in test_loader:
        outputs = model(input)
        outputs = outputs.reshape(-1).to(device)
        # outputs = torch.round(outputs)
        print(type(outputs))

        target = target.reshape(-1).to(device)
        # print(outputs)
        # print()
        # print(target)
        acc = accuracy20(outputs, target)
        total_acc += acc
        # max returns (value ,index)
        # _, predicted = torch.max(outputs.data, 1)
        # n_samples += labels.size(0)
        # n_correct += (predicted == labels).sum().item()
    mean_acc = total_acc / len(test_loader)
    # acc = 100.0 * n_correct / n_samples
    print(f'Accuracy: {mean_acc} %')


<class 'torch.Tensor'>
0/18
<class 'torch.Tensor'>
0/27
<class 'torch.Tensor'>
0/18
<class 'torch.Tensor'>
0/40
<class 'torch.Tensor'>
0/37
<class 'torch.Tensor'>
0/23
<class 'torch.Tensor'>
0/41
<class 'torch.Tensor'>
0/19
<class 'torch.Tensor'>
0/18
<class 'torch.Tensor'>
0/61
<class 'torch.Tensor'>
0/26
<class 'torch.Tensor'>
0/28
<class 'torch.Tensor'>
0/18
<class 'torch.Tensor'>
0/20
<class 'torch.Tensor'>
0/21
<class 'torch.Tensor'>
0/56
<class 'torch.Tensor'>
0/37
<class 'torch.Tensor'>
0/18
<class 'torch.Tensor'>
0/18
<class 'torch.Tensor'>
0/99
<class 'torch.Tensor'>
0/44
<class 'torch.Tensor'>
0/80
<class 'torch.Tensor'>
0/18
<class 'torch.Tensor'>
0/34
<class 'torch.Tensor'>
0/57
<class 'torch.Tensor'>
0/36
<class 'torch.Tensor'>
0/18
<class 'torch.Tensor'>
0/18
<class 'torch.Tensor'>
0/27
<class 'torch.Tensor'>
0/18
<class 'torch.Tensor'>
0/18
<class 'torch.Tensor'>
0/18
<class 'torch.Tensor'>
0/18
<class 'torch.Tensor'>
0/20
<class 'torch.Tensor'>
0/31
<class 'torch.Tensor

KeyboardInterrupt: 